In [54]:
import numpy as np
import feather as ftr
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

train = ftr.read_dataframe('../data/input/min_maxed_train.feather')
test = ftr.read_dataframe('../data/input/min_maxed_test.feather')


In [55]:
train=train.iloc[:300,:]
test=test.iloc[:300,:]

data = train.append(test)
data.drop(columns=['target', 'ID_code'], inplace=True)

kmf = KMeansFeaturizer()
k_data = kmf.fit_transform(data)

data = pd.DataFrame(k_data.toarray())
data = data.add_prefix('k_')

test = data.iloc[train.shape[0]:, :].reset_index(drop=True)
train = data.iloc[:train.shape[0], :].reset_index(drop=True)


c:\users\k-higuchi\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [61]:
train['k_var_2'].sum()

25.0

In [24]:
#big_diff
features = train.columns.values[2:202]
t0 = train.loc[train['target'] == 0]
t1 = train.loc[train['target'] == 1]
train_ID_df=train.iloc[:,:2]
test_ID_df=test.iloc[:,:1]

train.drop(columns=['target','ID_code'],inplace=True)
test.drop(columns=['ID_code'],inplace=True)


plus_idxs = (t1[features].mean(axis=0) - t0[features].mean(axis=0)
            ).sort_values(ascending=False)[0:10].index

minus_idxs = (t1[features].mean(axis=0) - t0[features].mean(axis=0)
            ).sort_values(ascending=False)[-10:].index

In [ ]:
#平均の差分が大きい特徴量を確認

idxs=(t1[features].mean(axis=0)-t0[features].mean(axis=0)).sort_values(ascending=False)[-10:].index

In [ ]:
(t1[features].mean(axis=0)-t0[features].mean(axis=0)).sort_values(ascending=False)[-10:]

In [32]:
class KMeansFeaturizer:
    """
    数値データをk-meansのクラスタIDに変換します。
    
    この変換器は入力データに対してk-meansを実行し、各データポイントを最も近い
    クラスタのIDに変換します。ターゲット変数yが存在する場合、クラス分類の境界を
    より重視したクラスタリングの結果を得るために、ターゲット変数をスケーリングして
    入力データに含めてk-meansに渡します。
    """
    #k=出力するクラスタの数
    def __init__(self, k=100, target_scale=5.0, random_state=None):
        self.k = k
        self.target_scale = target_scale
        self.random_state = random_state
        # np.array(range(k)).reshape(-1,1) ->2次元配列にそれぞれ一個ずつ要素が入っている。([[0],[1]...])
        #要素が入ってきたとき、one-hotvectorに変換する?([0]->[1,0,0...])
        self.cluster_encoder = OneHotEncoder(categories='auto').fit(np.array(range(k)).reshape(-1,1))
        
    def fit(self, X, y=None):
        """
        入力データに対しk-meansを実行し、各クラスタの中心を見つけます。
        """
        if y is None:
            # ターゲット変数がない場合、ふつうのk-meansを実行します。
            km_model = KMeans(n_clusters=self.k, n_init=20, random_state=self.random_state)
            km_model.fit(X)
            self.km_model = km_model
            self.cluster_centers_ = km_model.cluster_centers_
            return self

        # ターゲット変数がある場合。スケーリングして入力データに含めます。
        data_with_target = np.hstack((X, y[:,np.newaxis]*self.target_scale))

        # ターゲットを組み入れたデータで事前学習するためのk-meansモデルを構築します。
        km_model_pretrain = KMeans(n_clusters=self.k, n_init=20, random_state=self.random_state)
        km_model_pretrain.fit(data_with_target)

        # ターゲット変数の情報を除いて元の空間におけるクラスタを得るために
        # k-meansを再度実行します。事前学習で見つけたクラスタの中心を
        # 使って初期化し、クラスタの割り当てと中心の再計算を1回にします。
        km_model = KMeans(n_clusters=self.k, init=km_model_pretrain.cluster_centers_[:,:2], n_init=1, max_iter=1)
        km_model.fit(X)
        
        self.km_model = km_model
        self.cluster_centers_ = km_model.cluster_centers_
        return self
    
    def transform(self, X, y=None):
        """
        入力データポイントに最も近いクラスタのIDを返します。
        """
        clusters = self.km_model.predict(X)
        #clustersが[[0],[8]...]みたいに代入されてそれがワンホットベクトルとして変換されて出力される?
        return self.cluster_encoder.transform(clusters.reshape(-1,1))
    
    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X, y)

In [ ]:
train = ftr.read_dataframe('../data/input/train.feather')
test = ftr.read_dataframe('../data/input/test.feather')

#min_maxedを適用しておく。
features = train.columns.values[2:202]
target=train['target']
train_ID=train['ID_code']
test_ID=test['ID_code']

data_idx=data.index
data_col=data.columns

# data = train.append(test)
train.drop(columns=['target','ID_code'],inplace=True)
test.drop(columns=['ID_code'],inplace=True)

mm_scale = MinMaxScaler(copy=True, feature_range=(0, 1))
mm_scale.fit(train)
scale_train=mm_scale.transform(train)
scale_test=mm_scale.transform(test)

train=pd.DataFrame(scale_train,index=data_idx,columns=data_col)
test=pd.DataFrame(scale_train,index=data_idx,columns=data_col)

test = data.iloc[train.shape[0]:, :].reset_index(drop=True)
train = data.iloc[:train.shape[0], :].reset_index(drop=True)

train['target']=target
train['ID_code']=train_ID
test['ID_code']=test_ID

# train.to_feather('../data/input/min_maxed_train.feather')
# test.to_feather('../data/input/min_maxed_test.feather')

In [ ]:
features = train.columns.values[2:202]
t0 = train.loc[train['target'] == 0]
t1 = train.loc[train['target'] == 1]

plus_idxs = (t1[features].mean(axis=0) - t0[features].mean(axis=0)
            ).sort_values(ascending=False)[0:10].index

minus_idxs = (t1[features].mean(axis=0) - t0[features].mean(axis=0)
            ).sort_values(ascending=False)[-10:].index


In [ ]:

features = train.columns.values[2:202]
t0 = train.loc[train['target'] == 0]
t1 = train.loc[train['target'] == 1]



plus_idxs = (t1[features].mean(axis=0) - t0[features].mean(axis=0)
            ).sort_values(ascending=False)[0:10].index

minus_idxs = (t1[features].mean(axis=0) - t0[features].mean(axis=0)
            ).sort_values(ascending=False)[-10:].index

In [ ]:
print(train[minus_idxs].sum(axis=1))

In [ ]:
minus_idxs

In [ ]:
train=reduce_mem_usage(train, verbose=True)

In [21]:
train = ftr.read_dataframe('../data/input/train.feather')
test = ftr.read_dataframe('../data/input/test.feather')

train_ID_df=train.iloc[:,:2]
test_ID_df=test.iloc[:,:1]
features = train.columns.values[2:202]

train.drop(columns=['target','ID_code'],inplace=True)
test.drop(columns=['ID_code'],inplace=True)

mm_scale = MinMaxScaler(copy=True, feature_range=(0, 1))
train_scaled=mm_scale.fit_transform(train)
test_scaled=mm_scale.transform(test)

train_scaled_df=pd.DataFrame(train_scaled,columns=features)
test_scaled_df=pd.DataFrame(test_scaled,columns=features)

train_scaled_df=pd.concat([train_ID_df,train_scaled_df],axis=1)
test_scaled_df=pd.concat([test_ID_df,test_scaled_df],axis=1)

train_scaled_df.to_feather('../data/input/min_maxed_train.feather')
test_scaled_df.to_feather('../data/input/min_maxed_test.feather')
